In [18]:
import cantera as ct
import numpy as np
import pandas as pd
import concurrent.futures
import optunity

print(ct.__version__)

3.1.0a2


In [22]:
class Model:

    def __init__(self):
        self.T=383 # Initial temperature in K
        self.p=1 # pressure in bar
        self.volume= 0.1e-6 # volume of the catalyst zone
        self.beta=20.0 # temperature ramp in K/min
        self.mcat=0.040 # catalyst mass in g
        self.catarea = 10.4 # m2/g 
        #self.facet_fraction = [0.6923, 0.211, 0.0439, 0.0528, 0.0, 0.0, 0.0, 0.0] # [111, 100, 211, 110, edges...]
        self.facet_fraction = [0.7015, 0.2127, 0.0314, 0.0542, 0.0, 0.0, 0.0, 0.0] # [111, 100, 211, 110, edges...]
        self.sccm=50*1e-6/60 # Flow rate m3/s
        self.offset = 0

def TPD(model,k):
    try:
        yaml_file= '/users/bkreitz1/scratch/multifacet/TPD/multifacet.yaml'
        print(yaml_file)
        # import the gas model and set the initial conditions
        gas = ct.Solution(yaml_file, 'gas')
        gas.TPX = model.T, model.p*ct.one_atm, 'Ar:1, CO2:0.0, CO:0'

        # import the (111) surface
        surf111 = ct.Interface(yaml_file,'surface1', [gas])
        surf111.TP = model.T, model.p*ct.one_atm
        surf111.coverages = {'site(111)':1, 'O(111)':0, 'OC(111)':0 , 'CO2(111)':0}
    
        # import the (211) surface
        surf211 = ct.Interface(yaml_file,'surface2', [gas])
        surf211.TP = model.T, model.p*ct.one_atm
        surf211.coverages = {'site(211)':1, 'O(211)':0, 'OC(211)':0 , 'CO2(211)':0}

        #import the (100) surface
        surf100 = ct.Interface(yaml_file,'surface3', [gas])
        surf100.TP = model.T, model.p*ct.one_atm
        surf100.coverages = {'site(100)':1, 'O(100)':0, 'OC(100)':0 , 'CO2(100)':0}
    
        #import the (110) surface
        surf110 = ct.Interface(yaml_file,'surface4', [gas])
        surf110.TP = model.T, model.p*ct.one_atm
        surf110.coverages = {'site(110)':1, 'O(110)':0, 'OC(110)':0 , 'CO2(110)':0}
    

        edge1 = ct.Interface(yaml_file,'edge12', [gas, surf111, surf211])
        edge1.TP = model.T, model.p*ct.one_atm
        edge2 = ct.Interface(yaml_file,'edge23', [gas, surf100, surf211])
        edge2.TP = model.T, model.p*ct.one_atm
        edge3 = ct.Interface(yaml_file,'edge14', [gas, surf111, surf110])
        edge3.TP = model.T, model.p*ct.one_atm
        edge4 = ct.Interface(yaml_file,'edge34', [gas, surf100, surf110])
        edge4.TP = model.T, model.p*ct.one_atm
            
        # Normalize
        #model.facet_fraction=[0.69984177,0.25254703,0.02550416,0.02210705,0., 0.,  0., 0.] # [111, 100, 211, 110, edges...]
        normalized_facet_fraction = model.facet_fraction/np.sum(model.facet_fraction)
    
        # Cantera setup
        upstream = ct.Reservoir(gas, name='upstream')
        downstream = ct.Reservoir(gas, name='downstream')
    
        r=ct.IdealGasReactor(gas, energy='off')
        r.volume=model.volume
        rsurf111=ct.ReactorSurface(surf111, r, A=normalized_facet_fraction[0]*model.catarea*model.mcat)
        rsurf100=ct.ReactorSurface(surf100, r, A=normalized_facet_fraction[1]*model.catarea*model.mcat)
        rsurf211=ct.ReactorSurface(surf211, r, A=normalized_facet_fraction[2]*model.catarea*model.mcat)
        rsurf110=ct.ReactorSurface(surf110, r, A=normalized_facet_fraction[3]*model.catarea*model.mcat)
    
        rsurfEdge1 = ct.ReactorSurface(edge1, r, A=normalized_facet_fraction[4]*model.catarea*model.mcat)
        rsurfEdge2 = ct.ReactorSurface(edge2, r, A=normalized_facet_fraction[5]*model.catarea*model.mcat)
        rsurfEdge3 = ct.ReactorSurface(edge3, r, A=normalized_facet_fraction[6]*model.catarea*model.mcat)
        rsurfEdge4 = ct.ReactorSurface(edge4, r, A=normalized_facet_fraction[7]*model.catarea*model.mcat)
        
        # This is important since it adjusts the flow rates, 303 K is the temperature of the gas cylinders
        vdot = model.sccm  * ((ct.one_atm / gas.P) * ( 303 / 273.15)) # m^3/s 
        mass_flow_rate =  vdot* gas.density
        mflow = ct.MassFlowController(upstream, r, mdot=mass_flow_rate)
        v = ct.PressureController(r, downstream, primary=mflow, K=1e-9)
    
        sim=ct.ReactorNet([r])

        # set relative and absolute tolerances on the simulation 
        # for working case with no surface diffusion: rtol= 1e-4, atol= 1e-10
        sim.rtol = 1.0e-6
        sim.atol = 1.0e-14
        sim.max_steps = 100000
        rxn_time=np.linspace(0,7500,7500)
    
        gas_mole_fracs=np.zeros([gas.n_species, len(rxn_time)])
        surf_fracs111=np.zeros([surf111.n_species, len(rxn_time)])
        surf_fracs100=np.zeros([surf100.n_species, len(rxn_time)])
        surf_fracs211=np.zeros([surf211.n_species, len(rxn_time)])
        surf_fracs110=np.zeros([surf110.n_species, len(rxn_time)])

        co2_rop_111=np.zeros(len(rxn_time))
        co2_rop_211=np.zeros(len(rxn_time))
        co2_rop_110=np.zeros(len(rxn_time))
        co2_rop_100=np.zeros(len(rxn_time))
        co_rop_111=np.zeros(len(rxn_time))
        co_rop_211=np.zeros(len(rxn_time))
        co_rop_110=np.zeros(len(rxn_time))
        co_rop_100=np.zeros(len(rxn_time))
        
        Ti=np.zeros(len(rxn_time))
        sim.max_time_step = 1e-4
        nv = sim.n_vars
        for i in range(len(rxn_time)):
            time=rxn_time[i]
            print(time)
            if time < 10:
                sim.max_time_step = 1e-4
                upstream.thermo.TPX=model.T,model.p*ct.one_atm, 'Ar:1, He:0, CO2:0'
                upstream.syncState()
            elif (time > 10) and (time < 3610):  
                sim.max_time_step = 1e-2
                upstream.thermo.TPX=model.T,model.p*ct.one_atm, 'Ar:0, He:0.9013, CO2:0.0987'
                upstream.syncState()
            elif (time > 3610) and (time < 3970): 
                sim.max_time_step = 2
                upstream.thermo.TPX=model.T,model.p*ct.one_atm, 'Ar:1, He:0, CO2:0.0'
                upstream.syncState()
            elif (time > 3970) and (time < 5410):
                upstream.thermo.TPX=323,model.p*ct.one_atm, 'Ar:1, He:0, CO2:0'   
                upstream.syncState()
                r.thermo.TP=323,ct.one_atm
                r.syncState()
            else:
                sim.max_time_step = 1e-2  
                r.thermo.TP=323+model.beta/60.0*(rxn_time[i]-5410),model.p*ct.one_atm
                r.syncState()
        
            sim.reinitialize
            sim.advance(time)
            Ti[i]=r.thermo.T

            co2_rop_111[i] = rsurf111.kinetics.net_production_rates[7]
            co_rop_111[i] = rsurf111.kinetics.net_production_rates[8]
            co2_rop_211[i] = rsurf211.kinetics.net_production_rates[7]
            co_rop_211[i] = rsurf211.kinetics.net_production_rates[8]
            co2_rop_100[i] = rsurf100.kinetics.net_production_rates[7]
            co_rop_100[i] = rsurf100.kinetics.net_production_rates[8]
            co2_rop_110[i] = rsurf110.kinetics.net_production_rates[7]
            co_rop_110[i] = rsurf110.kinetics.net_production_rates[8]

            gas_mole_fracs[:,i]=gas.X
            surf_fracs111[:,i]=rsurf111.kinetics.X 
            surf_fracs100[:,i]=rsurf100.kinetics.X 
            surf_fracs211[:,i]=rsurf211.kinetics.X 
            surf_fracs110[:,i]=rsurf110.kinetics.X
            
        output_filename= '/users/bkreitz1/scratch/multifacet/TPD/tpd_cov_diff_raffaele.txt'   
        data=np.c_[Ti,gas_mole_fracs.T,surf_fracs111.T,surf_fracs100.T,surf_fracs110.T,surf_fracs211.T,
                   co2_rop_111.T, co_rop_111.T,co2_rop_100.T, co_rop_100.T,co2_rop_211.T, co_rop_211.T,co2_rop_110.T, co_rop_110.T]
        names=list(['Temp'] + gas.species_names+surf111.species_names+surf100.species_names+surf110.species_names+surf211.species_names +
                  ['co2_rop_111','co_rop_111','co2_rop_100','co_rop_100','co2_rop_211','co_rop_211','co2_rop_110','co_rop_110'])
        df=pd.DataFrame(data, columns=[names])
        df.to_csv(output_filename, index=False) 
    except:
        print("Something is wrong for " + str(k+model.offset))
    
    return
    

In [23]:
test = Model()
TPD(test,0)

/users/bkreitz1/scratch/multifacet/TPD/multifacet.yaml
0.0
1.0001333511134818
2.0002667022269636
3.0004000533404453
4.000533404453927
5.0006667555674085
6.000800106680891
7.000933457794373
8.001066808907854
9.001200160021336
10.001333511134817
11.0014668622483
12.001600213361781
13.001733564475263
14.001866915588746
15.002000266702227
16.00213361781571
17.00226696892919
18.00240032004267
19.002533671156154
20.002667022269634
21.002800373383117
22.0029337244966
23.00306707561008
24.003200426723563
25.003333777837046
26.003467128950525
27.00360048006401
28.00373383117749
29.00386718229097
30.004000533404454
31.004133884517934
32.00426723563142
33.0044005867449
34.00453393785838
35.00466728897186
36.00480064008534
37.004933991198826
38.00506734231231
39.00520069342579
40.00533404453927
41.00546739565275
42.005600746766234
43.00573409787972
44.0058674489932
45.00600080010668
46.00613415122016
47.00626750233364
48.006400853447126
49.00653420456061
50.00666755567409
51.00680090678757
52.0069